In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import os
sys.path.insert(0,"/home/daqtest/Processor/sandpro")
import sandpro
import configparser
import json
import scipy.stats
from matplotlib.colors import LogNorm

from scipy.optimize import curve_fit
import datetime
import pandas as pd
# %run ../WaveformProcessor.py
# %run ../FastProcessing.py
# %run ../FitSPE.py

sys.path.insert(0,"../src/")
import common.d2d as d2d
import common.utils as util
import data_processing.fast_processor as fast_processor

In [ ]:
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/noise_time_evolution_before_threshold_calibration_implemented/"
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/20240819_T102_50V_5.0sig/"
# paths = ["/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20241011_T98_45V_4.0sig"]
LXe_path = "/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20241010_2_T98_47V_5.5sig"

#LXe paths
# paths = ["/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20241010_2_T98_47V_5.5sig"]
paths = ["/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20241011_T98_45V_4.0sig"]
# paths = ["/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20241014_2_T98_47V_1.5sig"]

#Gas paths
paths.append("/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20240912_T102_51V_8.0sig")
paths.append("/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20240924_T102_48V_5.5sig")
paths.append("/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20240923_T102_49V_6.5sig")
paths.append("/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20240920_T102_49V_6.5sig")
# GXe_path = "/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/gain_run_1/T98_47V/"
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/T100_47_5,5sig_V/" # check waveform

plot = True # if want to replot the waveforms
# ignore_channel_list = np.array([2,8,11,14,20,23])
ignore_channel_list = np.array([])
channels = range(0,24) # or a list of channels
FastProcessorList = []
run_tags = []
 
# ##########################
# Load the meta data files

def find_meta_data_files(data_folder):
    
    meta_data_list = glob.glob(f"{data_folder}/*meta*")

    # Sort the file paths based on the extracted date and time in ascending order (oldest to newest)
    new_meta_data_list = sorted(meta_data_list, key=util.extract_date_meta_data)
    new_meta_data_list = sorted(new_meta_data_list[-24:], key=util.extract_channel_meta_data)
    print(new_meta_data_list)

    ## select channels
    selected_list = []
    for i in range(len(new_meta_data_list)):
        selected_list.append(meta_data_list[i])

    return selected_list

In [ ]:
height_V_gas = [np.array([])] * len(channels)
area_Vns_gas = [np.array([])] * len(channels)
rise_time_ns_gas = [np.array([])] * len(channels)

height_V_liquid = [np.array([])] * len(channels)
area_Vns_liquid = [np.array([])] * len(channels)
rise_time_ns_liquid = [np.array([])] * len(channels)

run_tags_cluster = ['GXe/gain_calibration', 'LXe/gain_calibration']

### Stack gas events
for path in paths:
    meta_data_files = find_meta_data_files(path)
    FastProcessor = fast_processor.FastProcessor(path, meta_data_files, 
                ignore_channel_list=ignore_channel_list)
    FastProcessor.process_runs()

    run_tag = FastProcessor.get_run_tag(meta_data_files[0])

    if run_tag == run_tags_cluster[0]:

        for channel, fast_info in enumerate(FastProcessor.list_of_fast_info):
            height_V_gas[channel] = np.concatenate((height_V_gas[channel], fast_info.heights_V))
            area_Vns_gas[channel] = np.concatenate((area_Vns_gas[channel], fast_info.areas_Vns))
            rise_time_ns_gas[channel] = np.concatenate((rise_time_ns_gas[channel], fast_info.rise_time_ns))

    elif run_tag == run_tags_cluster[1]:

        for channel, fast_info in enumerate(FastProcessor.list_of_fast_info):
            height_V_liquid[channel] = np.concatenate((height_V_liquid[channel], fast_info.heights_V))
            area_Vns_liquid[channel] = np.concatenate((area_Vns_liquid[channel], fast_info.areas_Vns))
            rise_time_ns_liquid[channel] = np.concatenate((rise_time_ns_liquid[channel], fast_info.rise_time_ns))

    run_tags.append(run_tag)


In [ ]:
### Stack GXe
### Comparison
plt.close()

# color_maps = [plt.cm.plasma, plt.cm.Dark2]
colors_dict = {'GXe/gain_calibration':'YlOrRd',
          'LXe/gain_calibration':'Blues'}
height_V = [height_V_gas, height_V_liquid]
area_Vns = [area_Vns_gas, area_Vns_liquid]
rise_time_ns = [rise_time_ns_gas, rise_time_ns_liquid]

fig, axes = plt.subplots(3,8,figsize=(35,15))

for run_mode, run_tag in enumerate(run_tags_cluster):
    assert(len(FastProcessor.list_of_fast_info) == 24)

    for channel in range(len(FastProcessor.list_of_fast_info)):
        
        plot_row = channel % 3
        plot_col = channel // 3

        # change the rows so that it match with the physical layout of the channels
        if plot_row == 0:
            plot_row = 1
        elif plot_row == 1:
            plot_row = 0

        mask = rise_time_ns[run_mode][channel] > 0.0

        # axes[plot_row,plot_col].hist2d(rise_time_ns,bins=5,range=[0,20],color='blue',alpha=0.5)
        axes[plot_row,plot_col].hist2d(area_Vns[run_mode][channel][mask],rise_time_ns[run_mode][channel][mask],
                                bins=[100,int(100/4)],
                                # range=[[-0.1,800],[0,400]],
                                range=[[-0.1,100],[0,100]],
                                cmap=colors_dict[run_tag],
                                norm=LogNorm())
        
        axes[plot_row,plot_col].set_title(f"Channel {fast_info.channel}")

# set the labels
for ax in axes.flat:
    ax.set(xlabel='Area [V*ns]', ylabel='Rise time [ns]')

# set title
fig_title = ""
for path, run_tag in zip(paths, run_tags):
    fig_title = fig_title + f"Path: {path.split('/')[-1]} \t Tag: {run_tag} \t Color: {colors_dict[run_tag]}\n"
plt.suptitle(fig_title)

# Move title upwards
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.tight_layout()

plt.show()